In [1]:
import networkx as nx
import sys
from ruamel.yaml import YAML
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

In [2]:
from systemflow.node import *

In [3]:
class CollectImage(Mutate):
    def __init__(self, name: str = "CollectImage"):
        fields = []
        properties = []
        parameters = ["resolution x", "resolution y", "bit depth"]
        super().__init__(name, fields, properties, parameters)

    def transform(self, message: Message, component: Component) -> tuple[dict, dict, dict]:
        #access the required fields/properties/parameters
        n_px_x = component.parameters["resolution x"]
        n_px_y = component.parameters["resolution y"]
        resolution = (n_px_x,
                      n_px_y,
                      component.parameters["bit depth"],)
        n_bytes = np.prod(resolution) / 8.0
        sample_rate = component.parameters["sample rate"]
        time = 0.0
        sensor_power = component.parameters["pixel energy"] * n_px_x * n_px_y

        #create the new fields in the message

        msg_fields = {"image data": n_bytes,
                  "time": time}
        msg_props = {"resolution": resolution,
                     "sample rate": sample_rate,}

        #create the new properties in the host
        host_props = {"sensor power": sensor_power,}

        return msg_fields, msg_props, host_props
       

In [4]:
ci = CollectImage()

In [5]:
ci_host = Component("Image sensor",
                    [CollectImage(),],
                    {"resolution x": 6000,
                     "resolution y": 4000,
                     "bit depth": 8,
                     "sample rate": 1000,
                     "pixel energy": 1e-6,},
                     {})

In [6]:
msg0 = Message({}, {})

In [7]:
ci_host.parameters

{'resolution x': 6000,
 'resolution y': 4000,
 'bit depth': 8,
 'sample rate': 1000,
 'pixel energy': 1e-06}

In [8]:
image_data, sensor_props = ci(msg0, ci_host)

In [9]:
image_data

Message(fields={'image data': np.float64(24000000.0), 'time': 0.0}, properties={'resolution': (6000, 4000, 8), 'sample rate': 1000})

In [10]:
class CollectTemperature(Mutate):
    def __init__(self, name: str = "CollectTemperature"):
        fields = []
        properties = []
        parameters = ["bit depth", "sample rate", "sensor power"]
        super().__init__(name, fields, properties, parameters)

    def transform(self, message: Message, component: Component) -> tuple[dict, dict, dict]:
        #access the required fields/properties/parameters
        n_bytes = component.parameters["bit depth"] / 8.0
        sample_rate = component.parameters["sample rate"]
        time = 0.0
        sensor_power = component.parameters["sensor power"]

        #create the new fields in the message
        msg_fields = {"temperature data": n_bytes,
                  "time": time}
        msg_props = {"sample rate": sample_rate,}

        #create the new properties in the host
        host_props = {"sensor power": sensor_power,}

        return msg_fields, msg_props, host_props
       

In [11]:
ct = CollectTemperature()

In [12]:
ct_host = Component("Thermocouple",
                    [CollectTemperature(),],
                    {"bit depth": 8,
                     "sample rate": 1200,
                     "sensor power": 1e-3,},
                     {})

In [13]:
ct.host_parameters

['bit depth', 'sample rate', 'sensor power']

In [14]:
temp_data, thermo_props = ct(msg0, ct_host)

In [15]:
dm = OverwriteMerge()

In [16]:
dm([msg0, msg0])

Message(fields={}, properties={})

In [17]:
msg1 = Message({"spiciness": 1000}, {"seasoning": "hot sauce"})
msg2 = Message({"spiciness": 2}, {"seasoning": "black pepper", "dish": "pasta"}) 

In [18]:
set(msg2.fields.keys()).union(set(msg2.properties.keys()))

{'dish', 'seasoning', 'spiciness'}

In [19]:
dm([msg1, msg2])

No merge provided for spiciness, taking first value
No merge provided for seasoning, taking first value


Message(fields={'spiciness': 1000}, properties={'seasoning': 'hot sauce', 'dish': 'pasta'})

In [20]:
class MaxSpicyMerge(Merge):
    def __init__(self):
        super().__init__({"spiciness": np.max,}, {"seasoning": lambda x: x[1]})
    


In [21]:
msm = MaxSpicyMerge()

In [22]:
msm([msg1, msg2])

Message(fields={'spiciness': np.int64(1000)}, properties={'seasoning': 'black pepper', 'dish': 'pasta'})

In [23]:
image_data.fields

{'image data': np.float64(24000000.0), 'time': 0.0}

In [24]:
temp_data.fields

{'temperature data': 1.0, 'time': 0.0}

In [25]:
dm([image_data, temp_data])

No merge provided for time, taking first value
No merge provided for sample rate, taking first value


Message(fields={'temperature data': 1.0, 'image data': np.float64(24000000.0), 'time': 0.0}, properties={'sample rate': 1000, 'resolution': (6000, 4000, 8)})

In [26]:
class RateMerge(Merge):
    def __init__(self):
        super().__init__({"time": np.min,}, {"sample rate": np.min})

In [27]:
rm = RateMerge()

In [28]:
rm([image_data, temp_data])

Message(fields={'temperature data': 1.0, 'image data': np.float64(24000000.0), 'time': np.float64(0.0)}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8)})

In [29]:
"data" in "image data"

True

In [30]:
for (key, value) in msg1.fields.items():
    print(key, value)

spiciness 1000


In [31]:
class DataRate(Mutate):
    def __init__(self, name: str = "DataRate"):
        msg_fields = []
        msg_properties = []
        host_parameters = []
        super().__init__(name, msg_fields, msg_properties, host_parameters)

    def transform(self, message: Message, component: Component):
        total_data = 0.0
        for (key, value) in message.fields.items():
            if "data" in key and key != "total data":
                total_data += value

        new_msg_fields = {"total data": total_data}
        return new_msg_fields, {}, {}

In [32]:
class StorageRate(Mutate):
    def __init__(self, name: str = "StorageRate"):
        msg_fields = ["total data"]
        msg_properties = ["sample rate"]
        host_parameters = []
        super().__init__(name, msg_fields, msg_properties, host_parameters)

    def transform(self, message: Message, component: Component):
        storage_rate = message.fields["total data"] * message.properties["sample rate"]
        new_host_properties = {"storage rate": storage_rate}
        return {}, {}, new_host_properties

In [33]:
ms_host = Component("Merge/Store", [DataRate(), StorageRate()], {}, {}, RateMerge())

In [34]:
nodes = [ci_host, ct_host, ms_host]

In [35]:
[c.name for c in nodes]

['Image sensor', 'Thermocouple', 'Merge/Store']

In [36]:
links = [DefaultLink("Image -> Merge/Store",
                     "Image sensor",
                     "Merge/Store"),
         DefaultLink("Thermocouple -> Merge/Store", "Thermocouple", "Merge/Store"),]

In [37]:
links

In [38]:
ms_graph = ExecutionGraph("Sense/Merge/Store", nodes, links)

In [39]:
ms_graph

In [40]:
ms_graph.graph.edges

OutEdgeView([('Image sensor', 'Merge/Store'), ('Thermocouple', 'Merge/Store')])

In [41]:
ms_graph.graph.nodes["Merge/Store"]['ref']

In [42]:
ms_graph.root

'Merge/Store'

In [43]:
rp = ms_graph.get_predecessors(ms_graph.get_node(ms_graph.root))

In [44]:
rp

In [45]:
c1 = rp[0](ms_graph)

In [46]:
c1[1]

[]

In [47]:
c1[0].output_msg

Message(fields={'image data': np.float64(24000000.0), 'time': 0.0}, properties={'resolution': (6000, 4000, 8), 'sample rate': 1000})

In [48]:
c1[0].properties

{'sensor power': 24.0}

In [49]:
c2 = rp[1](ms_graph)

In [50]:
c2[0].output_msg

Message(fields={'temperature data': 1.0, 'time': 0.0}, properties={'sample rate': 1200})

In [51]:
c2[0].properties

{'sensor power': 0.001}

In [52]:
rn = ms_graph.get_node(ms_graph.root)

In [53]:
im = rn.merge([c1[0].output_msg, c2[0].output_msg])

In [54]:
im

Message(fields={'temperature data': 1.0, 'image data': np.float64(24000000.0), 'time': np.float64(0.0)}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8)})

In [55]:
rn(ms_graph)

(<systemflow.node.Component at 0x1079f6fb0>,
 [(<systemflow.node.Component at 0x1079f6500>, []),
  (<systemflow.node.Component at 0x1079f5f00>, [])])

In [56]:
list(flatten(rn(ms_graph)))

In [57]:
g2 = ms_graph()

In [58]:
g2.iteration

1

In [59]:
class Convolve(Mutate):
    def __init__(self, name: str = "Convolve"):
        fields = []
        properties = ["resolution"]
        parameters = ["kernel x", "kernel y", "filters"]
        super().__init__(name, fields, properties, parameters)

    def transform(self, message: Message, component: Component):
        #access the required fields/properties/parameters
        res = message.properties["resolution"]
        kernel_x = component.parameters["kernel x"]
        kernel_y = component.parameters["kernel y"]
        filters = component.parameters["filters"]

        #calculate the number of ops required for the kernel
        kernel_ops = kernel_x * kernel_y * filters
        steps_x = (res[0] - kernel_x) // kernel_x
        steps_y = (res[1] - kernel_y) // kernel_y
        kernel_repeats = steps_x * steps_y

        #calculate the number of ops required for the kernel
        transform_operations = kernel_ops * kernel_repeats

        msg_fields = {"features": np.prod((steps_x, steps_y, filters)),}
        msg_properties = {}
        component_properties = {"transform operations": transform_operations,}
        

        return msg_fields, msg_properties, component_properties
       

In [60]:
convolve_host = Component("Convolution", [Convolve,], {"kernel x": 3, "kernel y": 3, "filters": 8}, {})

In [61]:
cnv = Convolve()

In [62]:
g2.nodes[0].name

'Merge/Store'

In [63]:
exp_msg = g2.nodes[0].output_msg

In [64]:
exp_msg.fields

{'temperature data': 1.0,
 'image data': np.float64(24000000.0),
 'time': np.float64(0.0),
 'total data': np.float64(24000001.0)}

In [65]:
exp_msg.properties

{'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8)}

In [66]:
message, props = cnv(exp_msg, convolve_host)

In [67]:
message

Message(fields={'temperature data': 1.0, 'image data': np.float64(24000000.0), 'time': np.float64(0.0), 'total data': np.float64(24000001.0), 'features': np.int64(21301344)}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8)})

In [68]:
props

{'transform operations': 191712096}

In [69]:
from systemflow.classifier import *

In [70]:
gc = GaussianClassifier(3.0)

In [71]:
gc.solve_reduction([900, 100], 0.9)

In [72]:
gc.error_matrix

array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])

In [73]:

class GaussianClassify(Mutate):
    def __init__(self, name: str = "GaussianClassify"):
        fields = []
        properties = ["sample rate"]
        parameters = ["skill", "variance", "reduction"]
        super().__init__(name, fields, properties, parameters)

    def transform(self, message: Message, component: Component):
        #access the required fields/properties/parameters
        sample_rate = message.properties["sample rate"]
        skill = component.parameters["skill"]
        variance = component.parameters["variance"]
        reduction = component.parameters["reduction"]

        #calculate the error statistics
        falses = sample_rate * reduction
        trues = sample_rate - falses
        inputs = np.array([falses, trues])
        
        gc = GaussianClassifier(skill, varscale=variance)
        output = gc(inputs, reduction)

        msg_fields = {"contingency": output}
        msg_properties = {"error matrix": gc.error_matrix}
        component_properties = {}

        return msg_fields, msg_properties, component_properties

In [74]:
class ClassifiedStorageRate(Mutate):
    def __init__(self, name: str = "ClassifiedStorageRate"):
        msg_fields = ["total data",]
        msg_properties = ["error matrix",]
        host_parameters = []
        super().__init__(name, msg_fields, msg_properties, host_parameters)

    def transform(self, message: Message, component: Component):
        storage_rate = message.fields["total data"] * get_passed(message.fields["contingency"])
        new_host_properties = {"storage rate": storage_rate,}
        return {}, {}, new_host_properties

In [75]:
1000 * 0.1

100.0

In [76]:
gcf = GaussianClassify()

In [77]:
classify_host = Component(["Classify"], [GaussianClassify(), DataRate(), ClassifiedStorageRate()], {"skill": 3, "variance": 1, "reduction": 0.9}, {})

In [78]:
msg2, props2 = gcf(message, classify_host)

In [79]:
msg2.fields

{'temperature data': 1.0,
 'image data': np.float64(24000000.0),
 'time': np.float64(0.0),
 'total data': np.float64(24000001.0),
 'features': np.int64(21301344),
 'contingency': array([[882,  17],
        [ 17,  82]])}

In [80]:
msg2.properties

{'sample rate': np.int64(1000),
 'resolution': (6000, 4000, 8),
 'error matrix': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}

In [81]:
props2

{}

In [82]:
ClassifiedStorageRate()(msg2, classify_host)

(Message(fields={'temperature data': 1.0, 'image data': np.float64(24000000.0), 'time': np.float64(0.0), 'total data': np.float64(24000001.0), 'features': np.int64(21301344), 'contingency': array([[882,  17],
        [ 17,  82]])}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8), 'error matrix': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}),
 {'storage rate': array([4.08000017e+08, 1.96800008e+09])})

In [83]:
get_rejected(msg2.fields["contingency"])

array([882,  17])

In [84]:
get_passed(msg2.fields["contingency"])

array([17, 82])

In [85]:
nodes = [Component("Image sensor",
                    [CollectImage(),],
                    {"resolution x": 6000,
                     "resolution y": 4000,
                     "bit depth": 8,
                     "sample rate": 1000,
                     "pixel energy": 1e-6,},
                     {}),
        Component("Thermocouple",
                    [CollectTemperature(),],
                    {"bit depth": 8,
                     "sample rate": 1200,
                     "sensor power": 1e-3,},
                     {}),
        Component("Merge/Convolve",
                    [Convolve(),],
                    {"kernel x": 3, "kernel y": 3, "filters": 8},
                    {},
                    RateMerge()),
        Component("Classify",
                    [GaussianClassify(), DataRate(), ClassifiedStorageRate(),],
                    {"skill": 3, "variance": 1, "reduction": 0.9},
                    {}),]

In [86]:
links = [DefaultLink("Image -> Merge/Convolve",
                     "Image sensor",
                     "Merge/Convolve"),
         DefaultLink("Thermocouple -> Merge/Convolve", 
                     "Thermocouple",
                     "Merge/Convolve"),
        DefaultLink("Merge/Convolve -> Classify",
                    "Merge/Convolve",
                    "Classify"),]

In [87]:
scc_graph = ExecutionGraph("Sense/Convolve/Classify", nodes, links)

In [88]:
scc2 = scc_graph(True)

Executing on node  Classify
Executing on node  Merge/Convolve
Executing on node  Image sensor
Executing on node  Thermocouple


In [89]:
scc2.iteration

1

In [90]:
scc2.nodes[0].name

'Classify'

In [91]:
scc2.nodes[0].output_msg

Message(fields={'time': np.float64(0.0), 'image data': np.float64(24000000.0), 'temperature data': 1.0, 'features': np.int64(21301344), 'contingency': array([[882,  17],
       [ 17,  82]]), 'total data': np.float64(24000001.0)}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8), 'error matrix': array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])})

In [92]:
scc2.get_node("Classify").output_msg

Message(fields={'time': np.float64(0.0), 'image data': np.float64(24000000.0), 'temperature data': 1.0, 'features': np.int64(21301344), 'contingency': array([[882,  17],
       [ 17,  82]]), 'total data': np.float64(24000001.0)}, properties={'sample rate': np.int64(1000), 'resolution': (6000, 4000, 8), 'error matrix': array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])})

In [93]:
scc2.root_node.properties

{'storage rate': array([4.08000017e+08, 1.96800008e+09])}

In [94]:
scc2.get_all_node_parameters()

{'Classify': {'skill': 3, 'variance': 1, 'reduction': 0.9},
 'Merge/Convolve': {'kernel x': 3, 'kernel y': 3, 'filters': 8},
 'Image sensor': {'resolution x': 6000,
  'resolution y': 4000,
  'bit depth': 8,
  'sample rate': 1000,
  'pixel energy': 1e-06},
 'Thermocouple': {'bit depth': 8, 'sample rate': 1200, 'sensor power': 0.001}}

In [95]:
ap = scc2.get_all_node_properties()

In [96]:
scc2.get_node("Classify").output_msg.properties

{'sample rate': np.int64(1000),
 'resolution': (6000, 4000, 8),
 'error matrix': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}

In [97]:
ap

{'Classify': {'storage rate': array([4.08000017e+08, 1.96800008e+09])},
 'Merge/Convolve': {'transform operations': 191712096},
 'Image sensor': {'sensor power': 24.0},
 'Thermocouple': {'sensor power': 0.001}}

In [98]:
class ConvPower(Metric):
    def __init__(self):
        super().__init__("Convolution Power", 
                         {},
                         {},
                         {"Merge/Convolve": "transform operations"},)
        
    def metric(self, graph: ExecutionGraph):
        host_props = graph.get_all_node_properties()
        ops = host_props["Merge/Convolve"]["transform operations"]
        power = ops * 1e-12

        metrics = {"power": power,}
        return metrics
    

In [99]:
cp = ConvPower()

In [100]:
cp(scc2)

{'power': 0.000191712096}